# 06 - Model Training

Model Training: MultinomialNB, LinearSVC+SMOTE, RandomForest, SVM, LogisticRegression
Evaluates all models and saves them for deployment.

## Imports


In [1]:
import os
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.pipeline import Pipeline
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt

## Ensure directories exist and Load Dataset

In [3]:
os.makedirs("../models/final", exist_ok=True)
os.makedirs("../reports", exist_ok=True)

# Path
CLEANED_OUTPUT_LABELED_PATH = Path("../data/netflix_reviews_Cleaned.csv")

# Load
df = pd.read_csv(CLEANED_OUTPUT_LABELED_PATH)
print(f"✅ Loaded Cleaned_labeled dataset with {len(df)} rows")

✅ Loaded Cleaned_labeled dataset with 41238 rows
